In [ ]:
#final_data_processing

import pandas as pd
import random
path ='/Users/bohyenkang/Desktop/data/second_data/new_data.csv'
df = pd.read_csv(path)
df = df.drop(df.columns[[1,2]],axis=1)


df = df.rename(columns={'Unnamed: 0' : 'movie_id'})
df = df.rename(columns={'영화명' : 'title'})
df = df.rename(columns={'장르' : 'genre'})
df = df.rename(columns={'제작국가' : 'country'})
df = df.rename(columns={'감독' : 'director'})
df = df.rename(columns={'제작국가' : 'country'})
df = df.rename(columns={'평점' : 'avg_rating'})
df = df.rename(columns={'배우' : 'actor'})
df = df.rename(columns={'제작연도' : 'year'})

a = []
b = []

for i in range(len(df)):
    x = random.randint(1,1000)
    y = random.randint(1,5)
    a.append(x)
    b.append(y)

data = {
    'user_id':a,
    'rating': b
}

df4 = pd.DataFrame(data)  

new_df3 = pd.concat([df,df4],axis=1)

new_df3 = new_df3.drop(index=new_df3[new_df3['avg_rating']==10].index)
new_df3 = new_df3.drop(index=new_df3[new_df3['avg_rating']==0].index)

new_df3.to_csv('df1000.csv',index=True)

In [ ]:
import pandas as pd
data =pd.read_csv('/Users/bohyenkang/df1000.csv')
data =data.drop(data.columns[0],axis=1)

from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
# reader 생성
reader = Reader(rating_scale = (1,5))

# 데이터를 생성한다. 
a1 = data[["user_id", "movie_id", "rating"]]
data2 = Dataset.load_from_df(a1, reader)
train_data = data2.build_full_trainset()

# 모델 생성 및 학습
model = SVD()
model.fit(train_data)

In [ ]:
#예측할 사용자를 선정하도록 하자. 
#user_id가 2번인 유저의 영화 평가 데이터를 타겟으로 설정하자. 2
#2번 유저의 데이터를 새로운 데이터 프레임에 저장한다.
target_user_id = 2
target_user_data = data.query(f"user_id == {target_user_id}")
target_user_data

In [ ]:
#다음은 user_id가 2인 평가한 영화 번호와 평점을 가져온다. 
#iterrows()를 사용하면 칼럼의 값을 시리즈로 출력해줄 수 있다. 
a1 = target_user_data[ ["movie_id", "rating"] ]

# user_id가 4인 평점을 딕셔너리에 저장한다.
target_user_movie_rating_dict = {}

for idx, row in a1.iterrows() :
	# 영화 id를 가져온다.
    movie_id = row["movie_id"]
    target_user_movie_rating_dict[movie_id] = row["rating"]
    
target_user_movie_rating_dict

In [ ]:
#마지막으로 4번 사용자가 아직 보지 않은 영화의 인덱스를 가져온다.
test_data = []

a1 = list(target_user_movie_rating_dict.keys())

In [ ]:
#영화의 수만큼 반복하면서 for문을 돌리고 영화 id를 가져온다. 
for movie_id in data["movie_id"] : 
    if movie_id not in a1 :
    	# 현재 영화를 담는다.
        test_data.append((target_user_id, movie_id, 0))  # 튜플로 담는다.
        
test_data

In [ ]:
#Surprise 형식에 맞게 데이터가 준비되었다. 이제 점수를 예측하는 모델을 만들고 학습을 시킨다.
target_user_predictions = model.test(test_data)
#target_user_predictions
accuracy.rmse(predictions)

In [ ]:
#예상 영화 평점만 출력한다.
for uid, iid, r_ui, est, _ in target_user_predictions : 
	print(f"{iid} : {round(est, 1)}")

In [ ]:
# 데이터는 완성되었으니 이제 상위 10개 데이터만 보기 좋게 출력하도록 하자.
# 먼저 예측된 점수를 담을 딕셔너리를 생성하고, 원하는 데이터(movie_id, rating)만을 추출해 담는다.
predict_dict = {
	"movie_id" : [],
    "rating" : []
}

# 원하는 데이터 추출
for _, movie_id, _, rating, _ in target_user_predictions :
    predict_dict["movie_id"].append(movie_id)
    predict_dict["rating"].append(round(rating, 1))

# 데이터프레임 저장
predict_df = pd.DataFrame(predict_dict)
predict_df.sort_values(by = "rating", inplace=True, ascending=False)
predict_df

In [ ]:
#상위 10개 데이터만 추출한다. 
a2 = predict_df.head(10)
a2

In [ ]:
#영화 제목과 평점 정보를 출력하기 위해 rating_data의 인덱스 값과 movie_data의 index 값이 일치하는 것을 가져온다. 
for mid in a2["movie_id"] : 
	# 영화 제목을 가져온다.
    str_expr = 'movie_id =='+str(mid)
    idx1 = data.query(str_expr).index
    movie_title = data.loc[idx1, "title"]
    
    # 평점 정보를 가져온다.
    idx2 = a2.query(str_expr).index
    movie_rating = a2.loc[idx2, "rating"]
    
    print(f"영화 제목 : {movie_title.values[0]}")
    print(f"장르 : {data.loc[movie_title.index[0]][2]}")
    print(f"국가 : {data.loc[movie_title.index[0]][3]}")
    print(f"배우 : {data.loc[movie_title.index[0]][6]}")
    print(f"평균 평점 : {data.loc[movie_title.index[0]][5]}")
    print(f"예상 평점 : {movie_rating.values[0]}")

출처 : https://moojuksublime.tistory.com/13?category=1203704